In [3]:
# Toggles
USE_FEATURE_ENGINEERING = True
USE_TRANSFORMATION = True
USE_SCALER = True
USE_WEIGHTED_RMSE = True
USE_GBR_BIAS_CORRECTION = True
USE_SYNTHETIC_SAMPLING = False
USE_RFR_FEATURE_SELECTION = False

# Protected Columns
exclude_columns = ['reiwa_price', 'identifier', 'reiwa_is_sold']

In [4]:
import numpy as np
import pandas as pd
import xgboost as xgb
import json
from tqdm.notebook import tqdm
from scipy.stats import boxcox
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectFromModel, VarianceThreshold
from scipy.special import inv_boxcox
from sklearn.ensemble import GradientBoostingRegressor
from collections.abc import MutableMapping


# Function to flatten nested dictionaries
def flatten_dict(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = f"{parent_key}{sep}{k}" if parent_key else k
        if isinstance(v, MutableMapping):
            items.extend(flatten_dict(v, new_key, sep=sep).items())
        else:
            items.append((new_key, v))
    return dict(items)

# Load and process property data
with open('property_data.json', 'r') as file:
    property_data = json.load(file)
property_df = pd.DataFrame(property_data)
property_df.fillna(0, inplace=True)
property_df['identifier'] = range(1, len(property_df) + 1)

# Load and process suburb data
with open('suburb_data.json', 'r') as file:
    suburb_data = json.load(file)
required_suburbs = set(property_df['reiwa_suburb'])
filtered_suburb_data = {k: flatten_dict(v) for k, v in suburb_data.items() if k in required_suburbs}
suburb_df = pd.DataFrame.from_dict(filtered_suburb_data, orient='index').reset_index().rename(columns={'index': 'reiwa_suburb'})
suburb_df.fillna(0, inplace=True)

# Merge property and suburb data
df = pd.merge(property_df, suburb_df, on='reiwa_suburb', how='left')

# Clean and filter the merged dataframe
df = (
    df.drop_duplicates(subset='reiwa_listing_id', keep='first')
    .query('reiwa_suburb_interest_level.notnull()')
    .assign(reiwa_price=lambda x: x[['reiwa_listing_price', 'reiwa_price']].max(axis=1))
    .drop(columns=['reiwa_listing_price'])
    .query('osm_local_community_population != 0')
)

# Create total rooms count for various filtering tasks
df['total_rooms'] = df['reiwa_bedrooms'] + df['reiwa_bathrooms'] + df['reiwa_parking']

# Identify houses under 40 sqm and scale up their land area if type == land or rooms > 4 (because the realtor meant hectares)
df.loc[(df['reiwa_landsize'] < 40) & ((df['reiwa_house_type'] == 'Land') | (df['total_rooms'] > 4)), 'reiwa_landsize'] *= 10000

# Sanity check to filter out unrealistic properties
min_price_per_sqm = 1000
max_price_per_sqm = 100000
conditions = (
    (df['reiwa_price'] <= 10000000) &
    (df['reiwa_landsize'] > 25) & (df['reiwa_landsize'] <= 10000) &
    ((df['total_rooms'] == 0) | (df['reiwa_landsize'] / df['total_rooms'] >= 10)) &
    ((df['reiwa_bedrooms'] == 0) | (df['reiwa_price'] / df['reiwa_bedrooms'] >= 50000)) &
    ((df['total_rooms'] > 0) | ((df['reiwa_price'] / df['reiwa_landsize'] >= min_price_per_sqm) & 
                                (df['reiwa_price'] / df['reiwa_landsize'] <= max_price_per_sqm)))
)
df = df[conditions].drop(columns=['total_rooms'])

# Make 'identifier' the first column
cols = ['identifier'] + [col for col in df.columns if col != 'identifier']
df = df[cols]

# Create a copy of the unsold houses with identifier saved
df_target = df[df['reiwa_is_sold'] == False].copy()

# Fill NaNs with zero (we have many optional columns)
df = df.apply(lambda col: col.fillna(col.median()) if col.dtype != 'O' else col)

# One-hot encode categorical fields
df = pd.get_dummies(df, columns=['reiwa_agency_no', 'reiwa_suburb', 'reiwa_house_type', 'reiwa_local_government'])

# Drop rows where 'suburb_interest_level' is null (indicates no valid REIWA data)
df = df[df['reiwa_suburb_interest_level'].notnull()]

# Drop text-based fields that are not being encoded
text_fields = ['reiwa_address', 'reiwa_image_url', 'reiwa_details_url', 'reiwa_agency_name', 'scsa_school']
df = df.drop(columns=text_fields)



In [ ]:
if USE_FEATURE_ENGINEERING:
    # Create a ratio for crime against people vs. property
    df['crime_ratio'] = np.where((df['wapol_total_person_crime'] != 0) & (df['wapol_total_property_crime'] != 0), df['wapol_total_property_crime'] / df['wapol_total_person_crime'], 0)

    # Summarize total crime
    df['crime_per_capita'] = (df['wapol_total_property_crime'] + df['wapol_total_person_crime']) / df['abs_people']

    # Create affordability ratio for mortgages
    df['affordability_ratio'] = df['abs_median_monthly_mortgage_repayment'] / df['abs_median_weekly_household_income']

    # Convert local_dining and local_shop to per capita values
    df['dining_per_capita'] = df['osm_local_dining'] / df['osm_local_community_population']
    df['shop_per_capita'] = df['osm_local_shop'] / df['osm_local_community_population']

    # Interaction Features
    df['landsize_population_interaction'] = df['reiwa_landsize'] * df['osm_local_community_population']
    df['rooms_bathrooms_interaction'] = df['reiwa_bedrooms'] * df['reiwa_bathrooms']

    # Polynomial Features
    df['landsize_squared'] = df['reiwa_landsize'] ** 2
    df['distance_to_perth_cbd_squared'] = df['osm_distance_to_perth_cbd'] ** 2

    # Distance Ratios
    df['distance_airport_cbd_ratio'] = df['osm_distance_to_perth_airport'] / (df['osm_distance_to_perth_cbd'] + 1)
    df['distance_fuel_station_ratio'] = df['osm_nearest_fuel_station'] / (df['osm_distance_to_perth_cbd'] + 1)

    # Ratio of median weekly rent to median weekly household income
    df['rent_income_ratio'] = df['abs_median_weekly_rent'] / (df['abs_median_weekly_household_income'] + 1)

In [ ]:
#garbage_fields = ['reiwa_pets_allowed', 'abs_female_ratio', 'reiwa_floor_plan_count']
#df = df.drop(columns=garbage_fields)
def drop_low_variance_bools(df, threshold=10):
    """
    Drop integer columns that only contain values of 0 and 1 and have very few 1 records.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    threshold (int): The maximum number of '1' values allowed for the column to be dropped. Default is 10.

    Returns:
    pd.DataFrame: The dataframe with the specified columns dropped.
    """
    cols_to_drop = []
    for col in df.select_dtypes(include='bool').columns:
        if set(df[col].unique()).issubset({0, 1}):
            if df[col].sum() < threshold:
                cols_to_drop.append(col)
    
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped

#df = drop_low_variance_bools(df, threshold=10)
def drop_columns_by_prefix(df, prefixes):
    """
    Drop columns that begin with any of the specified prefixes.

    Parameters:
    df (pd.DataFrame): The input dataframe.
    prefixes (list of str): The list of prefixes to check.

    Returns:
    pd.DataFrame: The dataframe with the specified columns dropped.
    """
    cols_to_drop = [col for col in df.columns if any(col.startswith(prefix) for prefix in prefixes)]
    df_dropped = df.drop(columns=cols_to_drop)
    return df_dropped

prefixes = ["wapol_offences_"]
df = drop_columns_by_prefix(df, prefixes)

# Log our target variable (must take place before scaler)
df['reiwa_price'], boxcox_lambda = boxcox(df['reiwa_price'])

# Function to determine if a column should use square root transformation
def should_use_sqrt(col):
    unique_values = df[col].nunique()
    return unique_values <= 20

# List to track transformed columns
transformed_columns = []

# Apply transformations if the toggle is on
if USE_TRANSFORMATION:
    for col in df.columns:
        if col in exclude_columns or df[col].dtype == bool or set(df[col].unique()) == {0, 1}:
            continue

        # Debug: Print the minimum value of the column
        min_value = df[col].min()

        if (df[col] < 0).any():
            df[col] = df[col] * -1

        if should_use_sqrt(col):
            df[col] = np.sqrt(df[col])
        else:
            try:
                if (df[col] <= 0).any():
                    df[col] += abs(df[col].min()) + 1  # Shift to make all values positive
                df[col], _ = boxcox(df[col])
            except ValueError as e:
                continue

        # Add the column to the list of transformed columns
        transformed_columns.append(col)

# Apply scaling if the toggle is on
if USE_SCALER:
    # Initialize StandardScaler
    scaler = StandardScaler()
    # Apply StandardScaler only to the transformed columns
    df[transformed_columns] = scaler.fit_transform(df[transformed_columns])

In [1]:
def feature_selection_with_rfr(df, target, protected_columns, variance_threshold=0.01, feature_importance_threshold=0.01, n_estimators=50, max_depth=5):
    """
    Perform feature selection using Variance Threshold and Random Forest Regressor, retaining specified protected columns.
    
    Parameters:
    df (pd.DataFrame): The dataframe containing the data.
    target (str): The target column name.
    protected_columns (list): A list of columns that should not be dropped.
    variance_threshold (float): The threshold for variance to remove low-variance features.
    feature_importance_threshold (float): The threshold for feature importance. Features with importance below this value will be dropped.
    n_estimators (int): Number of trees in the forest.
    max_depth (int): Maximum depth of the tree.
    
    Returns:
    pd.DataFrame: The dataframe with unnecessary columns dropped.
    """
    # Separate features and target
    X = df.drop(columns=[target])
    y = df[target]
    
    # Apply Variance Threshold to reduce dimensionality
    vt = VarianceThreshold(threshold=variance_threshold)
    X_reduced = vt.fit_transform(X)
    
    # Get the reduced feature names
    reduced_features = X.columns[vt.get_support()]
    
    # Create a new dataframe with the reduced features
    X_reduced_df = pd.DataFrame(X_reduced, columns=reduced_features, index=X.index)
    
    # Initialize and fit Random Forest Regressor
    model = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, random_state=42, n_jobs=-1, verbose=1)
    model.fit(X_reduced_df, y)
    
    # Select features based on importance
    selector = SelectFromModel(model, threshold=feature_importance_threshold, prefit=True)
    selected_features = X_reduced_df.columns[(selector.get_support())]
    
    # Ensure protected columns are retained
    columns_to_keep = set(selected_features).union(set(protected_columns))
    
    # Ensure all columns to keep are in the dataframe
    columns_to_keep = [col for col in columns_to_keep if col in df.columns]
    
    # Drop unnecessary columns directly from the original dataframe
    columns_to_drop = set(df.columns) - set(columns_to_keep) - {target}
    df.drop(columns=columns_to_drop, inplace=True)
    
    return df

if USE_RFR_FEATURE_SELECTION:
    df = feature_selection_with_rfr(df, 'reiwa_price', exclude_columns, max_depth=None, n_estimators=200, feature_importance_threshold=0.005)


NameError: name 'USE_RFR_FEATURE_SELECTION' is not defined

In [9]:
# Assuming df is already loaded
# We only want to train on is_sold == True since we are predicting pre-sales
df_sold = df[df['reiwa_is_sold'] == True]

# We modify this dataframe later so we copy it to avoid warnings
df_unsold = df[df['reiwa_is_sold'] == False].copy()

# Split the sold data into features (X) and target variable (y), excluding 'identifier'
X = df_sold.drop(['reiwa_price', 'reiwa_is_sold', 'identifier'], axis=1)
y = df_sold['reiwa_price']

# Perform the train-test split first to avoid data leakage
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Prepare the unsold data for evaluation
X_unsold = df_unsold.drop(['reiwa_price', 'reiwa_is_sold', 'identifier'], axis=1)
y_unsold = df_unsold['reiwa_price']

# Function to create synthetic samples by adding noise
def create_synthetic_samples(X, y, n_samples, noise_level=0.01):
    synthetic_X = np.tile(X, (n_samples, 1))
    synthetic_y = np.tile(y, n_samples)
    noise = np.random.normal(0, noise_level, synthetic_X.shape)
    return synthetic_X + noise, synthetic_y

# Function to generate synthetic data within specified ranges
def generate_synthetic_data(X_train, y_train, value_ranges):
    X_resampled = X_train.values
    y_resampled = y_train.values

    for value_range, n_synthetic_samples in value_ranges:
        range_min, range_max = value_range
        indices = (y_train >= range_min) & (y_train < range_max)
        X_range = X_train[indices]
        y_range = y_train[indices]

        if len(y_range) > 0:
            X_synthetic, y_synthetic = create_synthetic_samples(X_range.values, y_range.values, n_synthetic_samples)
            X_resampled = np.vstack([X_resampled, X_synthetic])
            y_resampled = np.hstack([y_resampled, y_synthetic])

    return X_resampled, y_resampled

# Define the value ranges and the number of synthetic samples for each range
value_ranges = [
    ((0, 200000), 10),         # Low values
    ((1000000, 3000000), 20),  # High values
    ((3000000, 10000000), 30)    # Very high values
]

if USE_SYNTHETIC_SAMPLING:
    X_resampled, y_resampled = generate_synthetic_data(X_train, y_train, value_ranges)
else:
    X_resampled = X_train.values
    y_resampled = y_train.values

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_resampled, label=y_resampled, nthread=-1)
dtest = xgb.DMatrix(X_test.values, label=y_test.values, nthread=-1)
dunsold = xgb.DMatrix(X_unsold.values, label=y_unsold.values, nthread=-1)

# Custom callback to update tqdm progress bar
class TqdmCallback(xgb.callback.TrainingCallback):
    def __init__(self, total_rounds, early_stopping_rounds):
        self.pbar = tqdm(total=total_rounds, desc="Training Progress")
        self.early_stopping_rounds = early_stopping_rounds
        self.best_score = float("inf")
        self.best_iteration = 0
        self.stopping_counter = 0

    def after_iteration(self, model, epoch, evals_log):
        train_rmse = evals_log['train']['rmse'][-1]
        eval_rmse = evals_log['eval']['rmse'][-1]
        unsold_rmse = evals_log['unsold']['rmse'][-1]

        self.pbar.set_postfix({
            'train_rmse': f'{train_rmse:.5f}', 
            'eval_rmse': f'{eval_rmse:.5f}',
            'unsold_rmse': f'{unsold_rmse:.5f}',
            'best_iteration': self.best_iteration
        })
        self.pbar.update(1)

        # Early stopping logic
        if eval_rmse < self.best_score:
            self.best_score = eval_rmse
            self.best_iteration = epoch
            self.stopping_counter = 0
        else:
            self.stopping_counter += 1

        if self.stopping_counter >= self.early_stopping_rounds:
            self.pbar.set_postfix_str(f'Early stopping at iteration {self.best_iteration} with best score {self.best_score}')
            self.pbar.close()
            return True  # Return True to stop training
        return False

wrmse_low = np.percentile(y_train, 10)
wrmse_high = np.percentile(y_train, 90)
def weighted_rmse(preds, dtrain):
    y_true = dtrain.get_label()
    residuals = preds - y_true
    weights = np.where((y_true <= wrmse_low) | (y_true >= wrmse_high), 2.5, 1)
    weighted_residuals = weights * residuals
    grad = 2 * weighted_residuals
    hess = 2 * weights
    return grad, hess

# Train the model with early stopping and progress bar
total_rounds = 10000
early_stopping_rounds = 50
tqdm_callback = TqdmCallback(total_rounds, early_stopping_rounds)

evals = [(dtrain, 'train'), (dtest, 'eval'), (dunsold, 'unsold')]

# Define XGBoost parameters with increased regularization and early stopping
xgb_params = {
    'tree_method': 'hist',
    'eval_metric': 'rmse',
    'device': 'cuda',
    'learning_rate': 0.05,
    'max_depth': 9
}

if USE_WEIGHTED_RMSE:
    xgb_model = xgb.train(
        params=xgb_params, 
        dtrain=dtrain, 
        num_boost_round=total_rounds, 
        evals=evals, 
        obj=weighted_rmse, 
        callbacks=[tqdm_callback, xgb.callback.EarlyStopping(rounds=early_stopping_rounds)], 
        verbose_eval=0
    )
else:
    xgb_params['objective'] = 'reg:squarederror'
    xgb_model = xgb.train(
        params=xgb_params, 
        dtrain=dtrain, 
        num_boost_round=total_rounds, 
        evals=evals, 
        callbacks=[tqdm_callback, xgb.callback.EarlyStopping(rounds=early_stopping_rounds)], 
        verbose_eval=0
    )

xgb_test_predictions = xgb_model.predict(dtest)
xgb_mae = mean_absolute_error(y_test, xgb_test_predictions)
xgb_rmse = mean_squared_error(y_test, xgb_test_predictions, squared=False)

# Print results
print(f"XGBoost - MAE: {xgb_mae:.4f}, RMSE: {xgb_rmse:.4f}")

# Save the XGBoost model in JSON format
xgb_model.save_model('xgb_model.json')

Training Progress:   0%|          | 0/10000 [00:00<?, ?it/s]

XGBoost - MAE: 0.0003, RMSE: 0.0004


c:\Users\jorda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [10]:
if USE_GBR_BIAS_CORRECTION:
    # Calculate residuals on the resampled training data
    xgb_train_predictions = xgb_model.predict(dtrain)
    residuals_train = y_resampled - xgb_train_predictions

    # Reshape the resampled training target variable to be used with GBR
    y_resampled_reshaped = y_resampled.reshape(-1, 1)

    # Train a gradient boosting regressor on the resampled training residuals
    gbr = GradientBoostingRegressor(n_estimators=200, max_depth=5, random_state=42)
    gbr.fit(y_resampled_reshaped, residuals_train)

    # Predict residuals for the test data using the trained GBR
    y_test_reshaped = y_test.values.reshape(-1, 1)
    predicted_residuals_test = gbr.predict(y_test_reshaped)

    # Adjust the initial test predictions
    adjusted_predictions = xgb_test_predictions + predicted_residuals_test

    # Evaluate the adjusted predictions
    adjusted_mae = mean_absolute_error(y_test, adjusted_predictions)
    adjusted_rmse = mean_squared_error(y_test, adjusted_predictions, squared=False)
    print(f"Adjusted XGBoost - MAE: {adjusted_mae:.4f}, RMSE: {adjusted_rmse:.4f}")


Adjusted XGBoost - MAE: 0.0003, RMSE: 0.0004


c:\Users\jorda\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(


In [12]:
# Load the XGBoost model
xgb_model = xgb.Booster()
xgb_model.load_model('xgb_model.json')
dunsold = xgb.DMatrix(X_unsold)
xgb_predictions = xgb_model.predict(dunsold)

# Adjust the predictions using the previously fitted gradient boosting regressor model
if USE_GBR_BIAS_CORRECTION:
    unsold_actual_prices = xgb_predictions.reshape(-1, 1)
    unsold_predicted_residuals = gbr.predict(unsold_actual_prices)
    adjusted_xgb_predictions = xgb_predictions + unsold_predicted_residuals
else:
    adjusted_xgb_predictions = xgb_predictions

# Add adjusted predictions to the unsold data
df_unsold.loc[:, 'model_prediction'] = inv_boxcox(adjusted_xgb_predictions, boxcox_lambda)

# Merge the predictions with property_data
df_target_pred = pd.merge(property_df, df_unsold[['identifier', 'model_prediction']], on='identifier', how='left')

# Reorder columns to place price_prediction as the third column
cols = list(df_target_pred.columns)
prediction_index = cols.index('model_prediction')
if prediction_index != 2:
    cols.insert(2, cols.pop(prediction_index))
df_target_pred = df_target_pred[cols]

# Remove rows with no predicted price
rows_before = df_target_pred.shape[0]
df_target_cleaned = df_target_pred.dropna(subset=['model_prediction'])
rows_after = df_target_cleaned.shape[0]

# Print the number of rows removed
rows_removed = rows_before - rows_after
print(f"Number of rows removed: {rows_removed}")

# Convert the cleaned DataFrame to JSON format
df_target_cleaned_json = df_target_cleaned.to_dict(orient='records')

# Save the JSON to a file
with open('property_data_unsold_predictions.json', 'w') as json_file:
    json.dump(df_target_cleaned_json, json_file, indent=4)

# Display a few entries from the JSON file
df_target_cleaned_json[:5]


Number of rows removed: 150539


[{'reiwa_address': '27 Hovea Court, Morley',
  'reiwa_price': 664999,
  'model_prediction': 821222.1753269205,
  'reiwa_landsize': 710.0,
  'reiwa_latitude': -31.8722835,
  'reiwa_longitude': 115.9294993,
  'reiwa_bedrooms': 4,
  'reiwa_bathrooms': 2,
  'reiwa_parking': 1,
  'reiwa_house_type': 'House',
  'reiwa_image_url': 'https://imagecdn.reiwa.com.au/listing/25/4810225-01.jpg?maxwidth=800&maxheight=600&quality=80',
  'reiwa_details_url': '/27-hovea-court-morley-4810225/',
  'reiwa_is_sold': False,
  'reiwa_floor_plan_count': 1,
  'reiwa_agency_name': 'Century 21 Gold Key Realty',
  'reiwa_agency_no': 13603,
  'reiwa_pets_allowed': False,
  'reiwa_suburb': 'Morley',
  'reiwa_listing_price': 689000.0,
  'reiwa_listing_id': 11826361,
  'osm_local_community_population': 13075,
  'osm_local_community_dwellings': 5187,
  'osm_distance_to_perth_cbd': 10.87875397060591,
  'osm_distance_to_perth_airport': 8.177887230719264,
  'osm_nearest_fuel_station': 0.5615864844997465,
  'osm_nearest_bu